<a href="https://colab.research.google.com/github/ajaymurali1998/hello-world/blob/main/sequence_to_sequence_language_translator_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense
import numpy as np

batch_size=64 # Batch size for training data
epochs=100 # number of epochs to train for
latent_dim=256 # latent dimensionality of the encoding space
num_samples=10000 #no of samples to train for
# path to the data txt file on disk
data_path='fra.txt'

In [23]:
input_texts=[]
output_texts=[]
input_characters=set()
output_characters=set()
with open('fra.txt',mode='r',encoding='utf-8') as f:
  lines=f.readlines()
x=[] 
for line in lines[:num_samples]:
  x=line.split('\t')
  input_texts.append(x[0])
  target_text='\t'+x[1]+'\n'
  output_texts.append(target_text)
  for item in x[0]:
    input_characters.add(item)
  for item in target_text:
    output_characters.add(item)


In [24]:
input_characters=sorted(list(input_characters))
output_characters=sorted(list(output_characters))
num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(output_characters)
max_encoder_seq_length=max([len(word) for word in input_texts])
max_decoder_seq_length=max([len(word) for word in output_texts])

In [25]:
print("Number of samples: ",num_samples)
print("Number of unique input tokens: ",num_encoder_tokens)
print("Number of unique output tokens: ",num_decoder_tokens)
print("Max sequence length for inputs: ",max_encoder_seq_length)
print("Max sequence length for outputs: ",max_decoder_seq_length)

Number of samples:  10000
Number of unique input tokens:  71
Number of unique output tokens:  92
Max sequence length for inputs:  15
Max sequence length for outputs:  59


In [26]:
input_token_index={}
output_token_index={}
for i,char in enumerate(input_characters):
  input_token_index[char]=i
for i,char in enumerate(output_characters):
  output_token_index[char]=i

In [27]:
encoder_input_data=np.zeros((len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype='float32')
decoder_input_data=np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')
decoder_target_data=np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype='float32')

In [28]:
for i,(input_text,target_text) in enumerate(zip(input_texts,output_texts)):
  for t,char in enumerate(input_text):
    encoder_input_data[i,t,input_token_index[char]]=1
  encoder_input_data[i,t+1:,input_token_index[' ']]=1
  for t,char in enumerate(target_text):
    # decode_target_data is ahead of decoder_input_data by one timestep
    decoder_input_data[i,t,output_token_index[char]]=1
    if t>0:
      # decoder_target_data will be ahead by one timestep
      # and will not include start character
      decoder_target_data[i,t-1,output_token_index[char]]=1
  decoder_input_data[i,t+1:,output_token_index[' ']]=1
  decoder_target_data[i,t:,output_token_index[' ']]=1


In [29]:
encoder_input_data[0].shape

(15, 71)

In [30]:
# define an input sequence and process it
encoder_inputs=Input(shape=(None,num_encoder_tokens))
encoder=LSTM(latent_dim,return_state=True)
encoder_outputs,state_h,state_c=encoder(encoder_inputs)
# we discard encoder outputs and only keep the states
encoder_states=[state_h,state_c] 

In [31]:
#set up the decoder, using encoder states as initial state.
decoder_inputs=Input(shape=(None,num_decoder_tokens))
# we set up our decoder to return full output sequences
# and to return full states as well. we dont use the 
# return states in the training model, but we will use them in inference
decoder_lstm=LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs,_,_=decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense=Dense(num_decoder_tokens,activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)

In [32]:
# define the model that will turn
# encoder input_data & decoder_input_data into decoder_output_data
model=Model([encoder_inputs,decoder_inputs],decoder_outputs)

# run training
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'] )
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,
          batch_size=batch_size,epochs=epochs,validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 4s 19ms/step - loss: 1.1417 - accuracy: 0.7357 - val_loss: 1.0233 - val_accuracy: 0.7219
Epoch 2/100
125/125 [==============================] - 2s 13ms/step - loss: 0.8122 - accuracy: 0.7783 - val_loss: 0.8198 - val_accuracy: 0.7730
Epoch 3/100
125/125 [==============================] - 2s 13ms/step - loss: 0.6443 - accuracy: 0.8177 - val_loss: 0.6916 - val_accuracy: 0.8012
Epoch 4/100
125/125 [==============================] - 2s 14ms/step - loss: 0.5663 - accuracy: 0.8352 - val_loss: 0.6334 - val_accuracy: 0.8138
Epoch 5/100
125/125 [==============================] - 2s 13ms/step - loss: 0.5208 - accuracy: 0.8475 - val_loss: 0.6007 - val_accuracy: 0.8227
Epoch 6/100
125/125 [==============================] - 2s 13ms/step - loss: 0.4869 - accuracy: 0.8564 - val_loss: 0.5576 - val_accuracy: 0.8365
Epoch 7/100
125/125 [==============================] - 2s 13ms/step - loss: 0.4587 - accuracy: 0.8645 - val_loss: 0.5347 - val_accuracy:

In [33]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [39]:
reverse_input_char_index=dict((i,char) for char,i in input_token_index.items())
reverse_target_char_index=dict((i,char) for char,i in output_token_index.items())
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, output_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [40]:
for seq_index in range(100):
  # take one sequence(part of the training set)
  # for trying out decoding
  input_seq=encoder_input_data[seq_index:seq_index+1]
  decoded_sentence=decode_sequence(input_seq)
  print("-")
  print("Input_sequence:",input_texts[seq_index])
  print("Decoded_sentence:",decoded_sentence)

-
Input_sequence: Go.
Decoded_sentence: Bouge !

-
Input_sequence: Go.
Decoded_sentence: Bouge !

-
Input_sequence: Go.
Decoded_sentence: Bouge !

-
Input_sequence: Hi.
Decoded_sentence: Salut !

-
Input_sequence: Hi.
Decoded_sentence: Salut !

-
Input_sequence: Run!
Decoded_sentence: File !

-
Input_sequence: Run!
Decoded_sentence: File !

-
Input_sequence: Run!
Decoded_sentence: File !

-
Input_sequence: Run!
Decoded_sentence: File !

-
Input_sequence: Run!
Decoded_sentence: File !

-
Input_sequence: Run!
Decoded_sentence: File !

-
Input_sequence: Run!
Decoded_sentence: File !

-
Input_sequence: Run!
Decoded_sentence: File !

-
Input_sequence: Run.
Decoded_sentence: File !

-
Input_sequence: Run.
Decoded_sentence: File !

-
Input_sequence: Run.
Decoded_sentence: File !

-
Input_sequence: Run.
Decoded_sentence: File !

-
Input_sequence: Run.
Decoded_sentence: File !

-
Input_sequence: Run.
Decoded_sentence: File !

-
Input_sequence: Run.
Decoded_sentence: File !

-
Input_sequence: Ru